In [1]:
import numpy as np
import pandas as pd
import scipy.linalg

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms

import sys
print(sys.version)
sys.path.append('../')
import os
import copy

from models.powertrain.bounded_powertrain import Bounded_powertrain
from models.kinematic.ideal_diff_drive import Ideal_diff_drive
from models.learning.blr_slip import SlipBayesianLinearRegression, FullBodySlipBayesianLinearRegression
from models.learning.blr_slip_acceleration import SlipAccelerationBayesianLinearRegression, FullBodySlipAccelerationBayesianLinearRegression
from models.kinematic.ICR_based import *
from models.kinematic.Perturbed_unicycle import *
from models.kinematic.enhanced_kinematic import *


from util.transform_algebra import *
from util.util_func import *

from sklearn import linear_model

3.10.12 (main, Jun 11 2023, 05:26:28) [GCC 11.4.0]


In [3]:
# import slip dataset
# 
# dataset_path = '../data/ral2023_dataset/husky/boreal_snow/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_tile_inflated/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_left-deflated/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/boreal_snow/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_a/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_b/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/slip_dataset_all.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/slip_dataset_all.pkl'

# dataset_path = '../data/ral2023_dataset/husky/boreal_snow/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_tile_inflated/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/husky/grand_salon_left-deflated/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/boreal_snow/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_a/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/ga_hard_snow_b/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/acceleration_dataset.pkl'
# dataset_path = '../data/ral2023_dataset/warthog_wheels/gravel_1/acceleration_dataset.pkl'
dataset_path = '../data/ral2023_dataset/warthog_wheels/ice/acceleration_dataset.pkl'


full_dataset = pd.read_pickle(dataset_path)
full_dataset_length = len(full_dataset)
train_dataset = full_dataset[:int(full_dataset_length/2)]
validate_dataset = full_dataset[int(full_dataset_length/2):]

# print(train_dataset)
print(train_dataset['imu_yaw_0'])

0     -0.003884
1      2.347554
2      2.704010
3      2.726604
4      1.509394
         ...   
290   -0.383529
291   -0.378660
292   -0.912517
293   -1.282512
294   -1.262309
Name: imu_yaw_0, Length: 295, dtype: float64


In [4]:
# isolate steady-state data

# extract cmd_body_vel arrays (input arrays)

idd_body_vel_x_str_list = []
idd_body_vel_y_str_list = []
idd_body_vel_yaw_str_list = []
for i in range(0, 40):
    str_idd_vel_x_i = 'idd_vel_x_' + str(i)
    str_idd_vel_y_i = 'idd_vel_y_' + str(i)
    str_idd_vel_yaw_i = 'idd_vel_yaw_' + str(i)
    idd_body_vel_x_str_list.append(str_idd_vel_x_i)
    idd_body_vel_y_str_list.append(str_idd_vel_y_i)
    idd_body_vel_yaw_str_list.append(str_idd_vel_yaw_i)
idd_body_vel_x_array = train_dataset[idd_body_vel_x_str_list].to_numpy()
idd_body_vel_y_array = train_dataset[idd_body_vel_y_str_list].to_numpy()
idd_body_vel_yaw_array = train_dataset[idd_body_vel_yaw_str_list].to_numpy()

x_train = np.column_stack((idd_body_vel_x_array.flatten(), idd_body_vel_yaw_array.flatten()))

# extract icp vels

str_icp_vel_x_list = []
str_icp_vel_y_list = []
str_icp_vel_yaw_list = []
for i in range(0, 40):
    str_icp_vel_x_i = 'icp_vel_x_' + str(i)
    str_icp_vel_y_i = 'icp_vel_y_' + str(i)
    str_icp_vel_yaw_i = 'icp_vel_yaw_' + str(i)
    str_icp_vel_x_list.append(str_icp_vel_x_i)
    str_icp_vel_y_list.append(str_icp_vel_y_i)
    str_icp_vel_yaw_list.append(str_icp_vel_yaw_i)
icp_vel_x_array = train_dataset[str_icp_vel_x_list].to_numpy()
icp_vel_y_array = train_dataset[str_icp_vel_y_list].to_numpy()
icp_vel_yaw_array = train_dataset[str_icp_vel_yaw_list].to_numpy()

# extract body_vel_distruptions arrays (output arrays)

str_body_vel_disturption_x_list = []
str_body_vel_disturption_y_list = []
str_body_vel_disturption_yaw_list = []
for i in range(0, 40):
    str_body_vel_disturption_x_i = 'body_vel_disturption_x_' + str(i)
    str_body_vel_disturption_y_i = 'body_vel_disturption_y_' + str(i)
    str_body_vel_disturption_yaw_i = 'body_vel_disturption_yaw_' + str(i)
    str_body_vel_disturption_x_list.append(str_body_vel_disturption_x_i)
    str_body_vel_disturption_y_list.append(str_body_vel_disturption_y_i)
    str_body_vel_disturption_yaw_list.append(str_body_vel_disturption_yaw_i)
    
body_vel_disturption_x_array = train_dataset[str_body_vel_disturption_x_list].to_numpy()
body_vel_disturption_y_array = train_dataset[str_body_vel_disturption_y_list].to_numpy()
body_vel_disturption_yaw_array = train_dataset[str_body_vel_disturption_yaw_list].to_numpy()

y_train_longitudinal_slip = body_vel_disturption_x_array.flatten()
y_train_lateral_slip = body_vel_disturption_y_array.flatten()
y_train_angular_slip = body_vel_disturption_yaw_array.flatten()

# compute mean body vel disturbance for each steady-state window
n_windows = len(train_dataset)

steady_state_mask = train_dataset['steady_state_mask'].to_numpy() == True

steady_state_idd_body_vel_x = idd_body_vel_x_array[steady_state_mask]
steady_state_idd_body_vel_y = idd_body_vel_y_array[steady_state_mask]
steady_state_idd_body_vel_yaw = idd_body_vel_yaw_array[steady_state_mask]

steady_state_icp_body_vel_x = icp_vel_x_array[steady_state_mask]
steady_state_icp_body_vel_y = icp_vel_y_array[steady_state_mask]
steady_state_icp_body_vel_yaw = icp_vel_yaw_array[steady_state_mask]

steady_state_body_vel_disturption_x = body_vel_disturption_x_array[steady_state_mask]
steady_state_body_vel_disturption_y = body_vel_disturption_y_array[steady_state_mask]
steady_state_body_vel_disturption_yaw = body_vel_disturption_yaw_array[steady_state_mask]


In [5]:
# import BLR model


slip_input_dimensions = 2
a_param_init = 0
b_param_init = 0
param_variance_init = 999999999999999999999
variance_init = 1000000000

# robot = 'husky'
# robot = 'marmotte'
robot = 'warthog-track'
if robot=='husky':
    wheel_radius = 0.33 / 2
    baseline = 0.55
    dt=0.05
if robot=='marmotte':
    wheel_radius = 0.116
    baseline = 0.5927
    dt=0.05
if robot == 'warthog-track':
    wheel_radius = 0.3
    baseline = 1.1652
    dt = 0.05
kappa_param = 1

full_body_slip_blr = FullBodySlipBayesianLinearRegression(1, 1, 3, a_param_init, b_param_init, param_variance_init, variance_init, baseline, wheel_radius, dt, kappa_param)
# full_body_slip_blr = FullBodySlipAccelerationBayesianLinearRegression(2, 1, 4, a_param_init, b_param_init, param_variance_init, variance_init, baseline, wheel_radius, dt, kappa_param)

In [6]:
# extract training data

x_train = np.column_stack((steady_state_idd_body_vel_x.flatten(), steady_state_idd_body_vel_y.flatten(), steady_state_idd_body_vel_yaw.flatten()))
y_train = np.column_stack((steady_state_body_vel_disturption_x.flatten(), steady_state_body_vel_disturption_y.flatten(), steady_state_body_vel_disturption_yaw.flatten()))

print(y_train)

# x_train = np.column_stack((idd_body_vel_x_array.flatten(), idd_body_vel_y_array.flatten(), idd_body_vel_yaw_array.flatten()))
# y_train = np.column_stack((body_vel_disturption_x_array.flatten(), body_vel_disturption_y_array.flatten(), body_vel_disturption_yaw_array.flatten()))


[[-0.14475794 -0.01951786  1.65244576]
 [-0.1471258  -0.01953539  1.63741323]
 [-0.14957558 -0.01918451  1.6195676 ]
 ...
 [ 0.95489483 -0.79089775 -0.61043556]
 [ 1.00390846 -0.79119049 -0.60677524]
 [ 1.00390846 -0.79119049 -0.60464748]]


In [8]:
# train BLR models, print resulting weights

full_body_slip_blr.train_params(x_train, y_train)

print('weights_x : ', full_body_slip_blr.body_x_slip_blr.weights)
print('weights_y : ', full_body_slip_blr.body_y_slip_blr.weights)
print('weights_yaw : ', full_body_slip_blr.body_yaw_slip_blr.weights)

# slip only
# full_body_slip_blr.save_params('../data/ral2023_dataset/husky/boreal_snow/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/husky/grand_salon_tile_inflated/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/husky/grand_salon_left-deflated/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/boreal_snow/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/ga_hard_snow_a/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/ga_hard_snow_b/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/grand_salon_tile_b/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/trained_params/slip/blr/steady-state/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/warthog_wheels/gravel_1/trained_params/slip/blr/steady-state/')
full_body_slip_blr.save_params('../data/ral2023_dataset/warthog_wheels/ice/trained_params/slip/blr/steady-state/')

# slip with acceleration
# full_body_slip_blr.save_params('../data/ral2023_dataset/husky/boreal_snow/trained_params/slip_acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/husky/grand_salon_tile_inflated/trained_params/slip_acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/husky/grand_salon_left-deflated/trained_params/slip_acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/boreal_snow/trained_params/slip_acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/ga_hard_snow_a/trained_params/slip_acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/ga_hard_snow_b/trained_params/slip_acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/marmotte/grand_salon_tile_b/trained_params/slip_acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip_acceleration/blr/')
# full_body_slip_blr.save_params('../data/ral2023_dataset/warthog_tracks/grand-axe_crusted-snow/trained_params/slip_acceleration/blr/')


weights_x :  [[0.54307277]]
weights_y :  [[0.12891673]]
weights_yaw :  [[-0.00874772]
 [ 0.01337405]
 [ 0.47011076]]


In [32]:
# train BLR steady-state for every

steady_state_idd_body_vel_x_shuffled = copy.deepcopy(steady_state_idd_body_vel_x.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_idd_body_vel_x_shuffled)
steady_state_idd_body_vel_y_shuffled = copy.deepcopy(steady_state_idd_body_vel_y.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_idd_body_vel_y_shuffled)
steady_state_idd_body_vel_yaw_shuffled = copy.deepcopy(steady_state_idd_body_vel_yaw.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_idd_body_vel_yaw_shuffled)

steady_state_body_vel_disturption_x_shuffled = copy.deepcopy(steady_state_body_vel_disturption_x.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_body_vel_disturption_x_shuffled)
steady_state_body_vel_disturption_y_shuffled = copy.deepcopy(steady_state_body_vel_disturption_y.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_body_vel_disturption_y_shuffled)
steady_state_body_vel_disturption_yaw_shuffled = copy.deepcopy(steady_state_body_vel_disturption_yaw.flatten())
np.random.seed(12345)
np.random.shuffle(steady_state_body_vel_disturption_yaw_shuffled)

print(steady_state_idd_body_vel_x_shuffled.flatten())
print(steady_state_idd_body_vel_y_shuffled.flatten())
print(steady_state_idd_body_vel_yaw_shuffled.flatten())

# print(steady_state_body_vel_disturption_x_shuffled.flatten())

# saved_params_root_path = '../data/ral2023_dataset/husky/boreal_snow/trained_params/slip/blr/steady-state/growing_training_set/'
saved_params_root_path = '../data/ral2023_dataset/warthog_tracks/boreal_mud/trained_params/slip/blr/steady-state/growing_training_set/'
# saved_params_root_path = '../data/ral2023_dataset/marmotte/grand_salon_tile_b/trained_params/slip/blr/steady-state/growing_training_set/'

for i in range(1, steady_state_body_vel_disturption_x_shuffled.shape[0]):
    training_timesteps_path = saved_params_root_path + str(i) + '/'
    if not os.path.exists(training_timesteps_path):
        os.makedirs(training_timesteps_path)
    x_train = np.column_stack((steady_state_idd_body_vel_x_shuffled[:i], 
                               steady_state_idd_body_vel_y_shuffled[:i], 
                               steady_state_idd_body_vel_yaw_shuffled[:i]))
    y_train = np.column_stack((steady_state_body_vel_disturption_x_shuffled[:i], 
                               steady_state_body_vel_disturption_y_shuffled[:i], 
                               steady_state_body_vel_disturption_yaw_shuffled[:i]))
    
    # print(x_train)
    # print(y_train)
    full_body_slip_blr = FullBodySlipBayesianLinearRegression(1, 1, 3, a_param_init, b_param_init, param_variance_init, variance_init, baseline, wheel_radius, dt, kappa_param)
    full_body_slip_blr.train_params(x_train, y_train)
    full_body_slip_blr.save_params(training_timesteps_path)
    
print('weights_x : ', full_body_slip_blr.body_x_slip_blr.weights)
print('weights_y : ', full_body_slip_blr.body_y_slip_blr.weights)
print('weights_yaw : ', full_body_slip_blr.body_yaw_slip_blr.weights)

[-0.94208818  0.57916244 -0.23903035 ...  0.03640889  0.77900891
  0.6963964 ]
[0. 0. 0. ... 0. 0. 0.]
[ 0.65709031 -1.         -1.         ... -1.          0.73606235
 -1.        ]


KeyboardInterrupt: 

In [8]:
weights_x :  [[-0.4235368]]
weights_y :  [[0.12007774]]
weights_yaw :  [[-0.04434587]
 [-0.04762662]
 [ 0.71554438]]


<>:3: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
<>:3: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
/tmp/ipykernel_33914/759267929.py:3: SyntaxWarning: list indices must be integers or slices, not float; perhaps you missed a comma?
  weights_yaw :  [[-0.04434587]


TypeError: list indices must be integers or slices, not float